# Arrow example

This example illustrates how to convert Arrow data that contains power-grid-model data to NumPy structured arrays, which the power-grid-model requests.

It is by no means intended to provide complete documentation on the topic, but only to show how such conversions could be done.

This example uses `pyarrow.RecordBatch` to demonstrate zero copy operations. The user can choose a `pyarrow.Table` or other structures based on the requirement.

**NOTE:** To run this example, the optional `examples` dependencies are required:

```sh
pip install .[examples]
```

In [1]:
%%capture cap --no-stderr
from IPython.display import display
from typing import Iterable

from power_grid_model import (
    PowerGridModel,
    initialize_array,
    CalculationMethod,
    power_grid_meta_data,
    ComponentType,
    DatasetType,
    ComponentAttributeFilterOptions,
)
from power_grid_model.data_types import SingleColumnarData
import pyarrow as pa
import pandas as pd
import numpy as np

In [2]:
# A constant showing error message
ZERO_COPY_ERROR_MSG = "Zero-copy conversion requested, but the data types do not match."


## Model

For clarity, a simple network is created. More complex cases work similarly and can be found in the other examples:

```
node 1 ---- line 4 ---- node 2 ----line 5 ---- node 3
   |                       |                      |
source 6               sym_load 7             sym_load 8
```

## Single symmetric calculations

Construct the input data for the model and construct the actual model.

Arrow uses a columnar data format while the power-grid-model offers both: row based or columnar data format.
Because of this, the columnar data format of power-grid-model provides a zero-copy interface for Arrow data. This differs from the row-based data format, for which conversions always require a copy.

### List the power-grid-model data types

See which attributes exist for a given component and which data types are used

In [ ]:
node_input_dtype = initialize_array("input", "node", 0).dtype
line_input_dtype = initialize_array("input", "line", 0).dtype
source_input_dtype = initialize_array("input", "source", 0).dtype
asym_load_input_dtype = initialize_array("input", "asym_load", 0).dtype
print("node:", node_input_dtype)
print("line:", line_input_dtype)
print("source:", source_input_dtype)
print("asym_load:", asym_load_input_dtype)

The primitive types of each attribute in the arrow tables need to match to make the operation efficient.
A zero copy is not guaranteed if the data types from power_grid_meta_data / initialize_array are not used.
Note that the asymmetric type of attribute in power-grid-model has a shape of `(3,)` along with a specific type. These represent the 3 phases of electrical system.
Hence asymmetric attributes need to be handled specially. 

In this tutorial we use the respective primitive types for the symmetrical attributes and a `FixedSizeListArray` of the primitive types with length 3 for asymmetrical attributes. This results in them being stored as contigious memory which would enable zero copy conversion. There might be other ways to approach this problem too.

#### Creating a Schema

We can make the task of assigning types easier by creating a schema based on the `DatasetType` and `ComponentType` directly from `power_grid_meta_data`. 
They can then directly be used to construct the `RecordBatch`.

In [ ]:
def pgm_schema(dataset_type: DatasetType, component_type: ComponentType, attributes: Iterable[str] | None = None):
    schemas = []
    component_dtype = power_grid_meta_data[dataset_type][component_type].dtype
    for meta_attribute, (dtype, _) in component_dtype.fields.items():
        if attributes is not None and meta_attribute not in attributes:
            continue
        if dtype.shape == (3,):
            pa_dtype = pa.list_(pa.from_numpy_dtype(dtype.base), 3)
        else:
            pa_dtype = pa.from_numpy_dtype(dtype)
        schemas.append((meta_attribute, pa_dtype))
    return pa.schema(schemas)


print("-------node combined asym scehma-------")
print(pgm_schema(DatasetType.input, ComponentType.node))
print("-------asym load combined asym scehma-------")
print(pgm_schema(DatasetType.input, ComponentType.asym_load))

### Create the grid using Arrow tables

The [power-grid-model documentation on Components](https://power-grid-model.readthedocs.io/en/stable/user_manual/components.html) provides documentation on which components are required and which ones are optional.

Construct the Arrow data as a table with the correct headers and data types.

In [ ]:
nodes_dict = {"id": [1, 2, 3], "u_rated": [10500.0, 10500.0, 10500.0]}


lines_dict = {
    "id": [4, 5],
    "from_node": [1, 2],
    "to_node": [2, 3],
    "from_status": [1, 1],
    "to_status": [1, 1],
    "r1": [0.11, 0.15],
    "x1": [0.12, 0.16],
    "c1": [4.1e-05, 5.4e-05],
    "tan1": [0.1, 0.1],
    "r0": [0.01, 0.05],
    "x0": [0.22, 0.06],
    "c0": [4.1e-05, 5.4e-05],
    "tan0": [0.4, 0.1],
}

sources_dict = {"id": [6], "node": [1], "status": [1], "u_ref": [1.0]}

sym_loads_dict = {
    "id": [7, 8],
    "node": [2, 3],
    "status": [1, 1],
    "type": [0, 0],
    "p_specified": [1.0, 2.0],
    "q_specified": [0.5, 1.5],
}

nodes = pa.record_batch(nodes_dict, schema=pgm_schema(DatasetType.input, ComponentType.node, nodes_dict.keys()))
lines = pa.record_batch(lines_dict, schema=pgm_schema(DatasetType.input, ComponentType.line, lines_dict.keys()))
sources = pa.record_batch(sources_dict, schema=pgm_schema(DatasetType.input, ComponentType.source, sources_dict.keys()))
sym_loads = pa.record_batch(
    sym_loads_dict, schema=pgm_schema(DatasetType.input, ComponentType.sym_load, sym_loads_dict.keys())
)

nodes
# the record batches of the other components can be printed similarly

### Convert the Arrow data to power-grid-model input data

The Arrow record batch or tables can then be converted to row based data or columnar data.
Converting Arrow data to columnar NumPy arrays is recommended to leverage the columnar nature of Arrow data. 
This conversion can be done with zero-copy operations.

Similar approach be adopted by the user to convert to row based data.

```{note}
The option of `zero_copy_only` in the function below is added in this demo to verify no copies are made. Its usage is not mandatory to do zero copy conversion.
```

In [ ]:
def arrow_to_numpy(
    data: pa.RecordBatch, dataset_type: DatasetType, component_type: ComponentType, zero_copy_only: bool = False
) -> np.ndarray:
    """Convert Arrow data to NumPy data."""
    result = {}
    result_dtype = power_grid_meta_data[dataset_type][component_type].dtype
    for name, column in zip(data.column_names, data.columns):
        column_data = column.to_numpy(zero_copy_only=zero_copy_only)
        if zero_copy_only and column_data.dtype != result_dtype[name]:
            raise ValueError(ZERO_COPY_ERROR_MSG)
        result[name] = column_data.astype(dtype=result_dtype[name], copy=False)
    return result


node_input = arrow_to_numpy(nodes, DatasetType.input, ComponentType.node, zero_copy_only=True)
line_input = arrow_to_numpy(lines, DatasetType.input, ComponentType.line)
source_input = arrow_to_numpy(sources, DatasetType.input, ComponentType.source)
sym_load_input = arrow_to_numpy(sym_loads, DatasetType.input, ComponentType.sym_load)

node_input

### Construct the complete input data structure

In [7]:
input_data = {
    ComponentType.node: node_input,
    ComponentType.line: line_input,
    ComponentType.source: source_input,
    ComponentType.sym_load: sym_load_input,
}

In [8]:
# Optional: validate the input data
from power_grid_model.validation import validate_input_data

validate_input_data(input_data)

### Use the power-grid-model

The `output_component_types` argument is set to `ComponentAttributeFilterOptions.relevant` to given out columnar data.

For more extensive examples, visit the [power-grid-model documentation](https://power-grid-model.readthedocs.io/en/stable/index.html).


In [ ]:
# construct the model
model = PowerGridModel(input_data=input_data, system_frequency=50)

# run the calculation
sym_result = model.calculate_power_flow(output_component_types=ComponentAttributeFilterOptions.relevant)

# use pandas to tabulate and display the results
sym_node_result = sym_result[ComponentType.node]
display(pd.DataFrame(sym_node_result))

#### Convert the symmetric result to Arrow format

Converting symmetrical results is straightforward by using schema from [Creating Schema](#creating-a-schema)

In [ ]:
pa_sym_node_result = pa.record_batch(
    sym_node_result, schema=pgm_schema(DatasetType.sym_output, ComponentType.node, sym_node_result.keys())
)
pa_sym_node_result

## Single asymmetric calculations

Asymmetric calculations have a tuple of values for some of the attributes and are not easily convertible to record batches.
Instead, one can have a look at the individual components of those attributes and/or flatten the arrays to access all components.

### Asymmetric input

To illustrate the conversion, let's consider a similar grid but with asymmetric loads.

```
node 1 ---- line 4 ---- node 2 ----line 5 ---- node 3
   |                       |                      |
source 6              asym_load 7            asym_load 8
```

In [ ]:
asym_loads_dict = {
    "id": [7, 8],
    "node": [2, 3],
    "status": [1, 1],
    "type": [0, 0],
    "p_specified": [[1.0, 1.0e-2, 1.1e-2], [2.0, 2.5, 4.5e2]],
    "q_specified": [[0.5, 1.5e3, 0.1], [1.5, 2.5, 1.5e3]],
}

asym_loads = pa.record_batch(
    asym_loads_dict, schema=pgm_schema(DatasetType.input, ComponentType.asym_load, asym_loads_dict.keys())
)

asym_loads

In [ ]:
def arrow_to_numpy_asym(
    data: pa.RecordBatch, dataset_type: DatasetType, component_type: ComponentType, zero_copy_only: bool = False
) -> np.ndarray:
    """Convert asymmetric Arrow data to NumPy data.

    This function is similar to the arrow_to_numpy function, but also supports asymmetric data."""
    result = {}
    result_dtype = power_grid_meta_data[dataset_type][component_type].dtype

    for name in result_dtype.names:
        if name not in data.column_names:
            continue
        dtype = result_dtype[name]

        if len(dtype.shape) == 0:
            column_data = data.column(name).to_numpy(zero_copy_only=zero_copy_only)
        else:
            column_data = data.column(name).flatten().to_numpy(zero_copy_only=zero_copy_only).reshape(-1, 3)

        if zero_copy_only and column_data.dtype.base != dtype.base:
            raise ValueError(ZERO_COPY_ERROR_MSG)
        result[name] = column_data.astype(dtype=dtype.base, copy=False)
    return result


asym_load_input = arrow_to_numpy_asym(asym_loads, DatasetType.input, ComponentType.asym_load, zero_copy_only=True)

asym_load_input

### Use the power-grid-model in asymmetric calculations

In [ ]:
asym_input_data = {
    ComponentType.node: node_input,
    ComponentType.line: line_input,
    ComponentType.source: source_input,
    ComponentType.asym_load: asym_load_input,
}

validate_input_data(asym_input_data, symmetric=False)

# construct the model
asym_model = PowerGridModel(input_data=asym_input_data, system_frequency=50)

# run the calculation
asym_result = asym_model.calculate_power_flow(
    symmetric=False, output_component_types=ComponentAttributeFilterOptions.relevant
)

# use pandas to display the results, but beware the data types
pd.DataFrame(asym_result[ComponentType.node]["u_angle"])

### Convert asymmetric power-grid-model output data to Arrow output data

In [ ]:
def numpy_columnar_to_arrow(
    data: SingleColumnarData, dataset_type: DatasetType, component_type: ComponentType
) -> pa.RecordBatch:
    """Convert NumPy data to Arrow data."""
    # pa.record_batch.from_arrays(data, schema=pgm_schema(DatasetType.result, ComponentType.node))
    component_pgm_schema = pgm_schema(dataset_type, component_type, data.keys())
    pa_columns = {}
    for attribute, data in data.items():
        primitive_type = component_pgm_schema.field(attribute).type

        if data.ndim == 2 and data.shape[1] == 3:
            pa_columns[attribute] = pa.FixedSizeListArray.from_arrays(data.flatten(), type=primitive_type)
        else:
            pa_columns[attribute] = pa.array(data, type=primitive_type)
    return pa.record_batch(pa_columns, component_pgm_schema)


pa_asym_node_result = numpy_columnar_to_arrow(
    asym_result[ComponentType.node], DatasetType.asym_output, ComponentType.node
)

pa_asym_node_result

## Batch data

power-grid-model supports batch calculations by providing an `update_data` argument, as shown in [this example](https://power-grid-model.readthedocs.io/en/stable/examples/Power%20Flow%20Example.html#batch-calculation).

Both the `update_data` and the output result are similar to the `input_data` and output data in the above, except that they have another dimension representing the batch index: the first index in the NumPy structured arrays.

This extra index can be represented in Arrow using a [`RecordBatch`](https://arrow.apache.org/docs/cpp/api/table.html#two-dimensional-datasets) or using any other multi-index data format.